# create database

> Core database functionality for the keybindings_fps app. This module is meant to run once to create the database and tables. Don't run this module again if the database already exists, because it will drop the existing tables.

In [ ]:
#| default_exp create_db

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pathlib import Path
from fasthtml.common import *

In [ ]:
#| export
def get_project_root():
    """Get the project root directory from either notebook or module context"""
    try:
        # When running as a module
        return Path(__file__).parent.parent.parent
    except NameError:
        # When running in notebook
        return Path.cwd().parent

In [ ]:
#| hide
get_project_root()

Path('/home/jelle/code/keybindings_fps')

In [ ]:
#|export
def init_db(data_dir=None):
    """Initialize the database connection
    Args:
        data_dir: Optional path to data directory. If None, uses project's data dir
    """
    if data_dir is None:
        data_dir = get_project_root() / 'data'
    data_dir.mkdir(exist_ok=True)
    return database(data_dir / 'game_bindings.db')

In [ ]:
#| hide
db = init_db()

In [ ]:
#|export
def create_tables(db):
    """Create all required database tables"""
    tables = ['categories', 'actions', 'games', 'game_keys', 'modifiers', 'bindings']
    
    # Drop tables if they exist
    for table in tables:
        if table in db.t:
            db.t[table].drop()
    
    # Create categories table
    db.t.categories.create(
        id=int,
        name=str,
        description=str,
        pk='id',
        not_null=['name'],
        replace=True
    )
    
    # Create actions table
    db.t.actions.create(
        id=int,
        name=str,
        description=str,
        category_id=int,
        pk='id',
        not_null=['name', 'category_id'],
        replace=True
    )
    
    # Create games table
    db.t.games.create(
        id=int,
        name=str,
        game_type=str,
        image=bytes,
        pk='id',
        not_null=['name'],
        replace=True
    )
    
    # Create game_keys table
    db.t.game_keys.create(
        id=int,
        name=str,
        pk='id',
        not_null=['name'],
        replace=True
    )
    
    # Create modifiers table
    db.t.modifiers.create(
        id=int,
        name=str,
        pk='id',
        not_null=['name'],
        replace=True
    )
    
    # Create bindings table
    db.t.bindings.create(
        id=int,
        game_id=int,
        action_id=int,
        key_id=int,
        modifier_id=int,
        pk='id',
        not_null=['game_id', 'action_id', 'key_id', 'modifier_id'],
        replace=True
    )


In [ ]:
#| hide
create_tables(db)
db.t

### Open existing database in project data directory

In [ ]:
db = init_db()

In [ ]:
db.t.actions()

[]

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()